In [ ]:
!pip install -r '/content/drive/MyDrive/CMU/SP21/18786-IDL/11785-final-project/requirements.txt'

     |████████████████████████████████| 3.3MB 6.5MB/s 
     |████████████████████████████████| 50.4MB 109kB/s 
     |████████████████████████████████| 11.3MB 33.5MB/s 
     |████████████████████████████████| 27.4MB 111kB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 112kB 49.1MB/s 
  Created wheel for Pillow: filename=Pillow-4.1.1-cp37-cp37m-linux_x86_64.whl size=1009522 sha256=f2a69e4b7c567bb5386fdee686401da6bfd750e6e805f357e2ebc0f1770f1896
  Stored in directory: /root/.cache/pip/wheels/3f/f3/3e/f24b51fe136968f797933559011acd48b16708fc306c924770
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35416 sha256=ff0d788f45c8a7ef870041822e52a0b32d1657040c167684f037110b9d5b6f65
  Stored in directory: /root/.cache/pip/wheels/4b/f4/11/bc4166107c27f07fd7bba707ffcb439619197638a1ac986df3
Successfully built Pillow olefile
ERROR: scikit-image 0.16.2 has requirement pillow>=4.3.0, but you'll have pillow 4.1.1 which is inco

In [ ]:
!git clone --single-branch --branch main https://[GITHUB_TOKEN]@github.com/hans-lizihan/11785-project.git

Cloning into '11785-project'...
remote: Enumerating objects: 45727, done.
remote: Counting objects: 100% (14904/14904), done.
remote: Compressing objects: 100% (14896/14896), done.
remote: Total 45727 (delta 25), reused 14885 (delta 7), pack-reused 30823
Receiving objects: 100% (45727/45727), 1.92 GiB | 40.70 MiB/s, done.
Resolving deltas: 100% (85/85), done.
Checking out files: 100% (64479/64479), done.


In [ ]:
import imageio
import itertools
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
import cv2, os
from tqdm import tqdm
import os, time, pickle, argparse
import torch.optim as optim
from google.colab.patches import cv2_imshow
from pathlib import Path


In [ ]:
def pre_process(img_rgb):
  num_down = 2       # number of downsampling steps
  num_bilateral = 7  # number of bilateral filtering steps

  # downsample image using Gaussian pyramid
  img_color = img_rgb
  for _ in range(num_down):
    img_color = cv2.pyrDown(img_color)

  # repeatedly apply small bilateral filter instead of
  # applying one large filter
  for _ in range(num_bilateral):
    img_color = cv2.bilateralFilter(img_color, d=9,
                      sigmaColor=9,
                      sigmaSpace=7)

  # upsample image to original size
  for _ in range(num_down):
    img_color = cv2.pyrUp(img_color)

  # convert to grayscale and apply median blur
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
  img_blur = cv2.medianBlur(img_gray, 7)

  # detect and enhance edges
  img_edge = cv2.adaptiveThreshold(img_blur, 255,
                    cv2.ADAPTIVE_THRESH_MEAN_C,
                    cv2.THRESH_BINARY,
                    blockSize=5,
                    C=2)

  # convert back to color, bit-AND with color image
  img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)
  img_cartoon = cv2.bitwise_and(img_color, img_edge)

  # display
  #cv2_imshow(img_cartoon)
  return img_cartoon

In [ ]:
def edge_mask(img, line_size=7, blur_value=7):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray_blur = cv2.medianBlur(gray, blur_value)
  edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
  return edges

def color_quantization(img, k=9):
# Transform the image
  data = np.float32(img).reshape((-1, 3))

# Determine criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

# Implementing K-Means
  ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  center = np.uint8(center)
  result = center[label.flatten()]
  result = result.reshape(img.shape)
  return result

def pre_process2(img_rgb):
  line_size = 7
  blur_value = 7
  edges = edge_mask(img_rgb, line_size, blur_value)

  total_color = 9
  img = color_quantization(img_rgb, total_color)

  blurred = cv2.bilateralFilter(img, d=7, sigmaColor=200,sigmaSpace=200)
  cartoon = cv2.bitwise_and(blurred, blurred, mask=edges)
  #cv2_imshow(cartoon)
  return cartoon

In [ ]:
# img_rgb = cv2.imread("/content/11785-project/data/src_flame/test/18212223_2c45a53a4a.jpg")
# pre_process(img_rgb)
# pre_process2(img_rgb)

def preprocessFolder(src, tgt):
    Path(tgt).mkdir(parents=True, exist_ok=True)
    for filename in tqdm(os.listdir(src), total=len(os.listdir(src))):
        img = cv2.imread(os.path.join(src,filename))
        if img is not None:
            cartoon = pre_process2(img)
            cv2.imwrite(os.path.join(tgt, filename), cartoon)
            
            
preprocessFolder("/content/11785-project/data/src_flame/train/", "/content/11785-project/data/src_cartoonflame/train/")
preprocessFolder("/content/11785-project/data/src_flame/test/", "/content/11785-project/data/src_cartoonflame/test/")

100%|██████████| 9/9 [00:03<00:00,  2.42it/s]


In [ ]:
def data_load(path, subfolder, transform, batch_size, shuffle=False, drop_last=True):
    dset = datasets.ImageFolder(path, transform)
    ind = dset.class_to_idx[subfolder]

    n = 0
    for i in range(dset.__len__()):
        if ind != dset.imgs[n][1]:
            del dset.imgs[n]
            n -= 1

        n += 1

    return torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)

def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()

In [ ]:
class resnet_block(nn.Module):
    def __init__(self, channel, kernel, stride, padding):
        super(resnet_block, self).__init__()
        self.channel = channel
        self.kernel = kernel
        self.strdie = stride
        self.padding = padding
        self.conv1 = nn.Conv2d(channel, channel, kernel, stride, padding)
        self.conv1_norm = nn.InstanceNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, kernel, stride, padding)
        self.conv2_norm = nn.InstanceNorm2d(channel)

        initialize_weights(self)

    def forward(self, input):
        x = F.relu(self.conv1_norm(self.conv1(input)), True)
        x = self.conv2_norm(self.conv2(x))

        return input + x #Elementwise Sum
 

class generator(nn.Module):
    # initializers
    def __init__(self, in_nc, out_nc, nf=32, nb=6):
        super(generator, self).__init__()
        self.input_nc = in_nc
        self.output_nc = out_nc
        self.nf = nf
        self.nb = nb
        self.down_convs = nn.Sequential(
            nn.Conv2d(in_nc, nf, 7, 1, 3), #k7n64s1
            nn.InstanceNorm2d(nf),
            nn.ReLU(True),
            nn.Conv2d(nf, nf * 2, 3, 2, 1), #k3n128s2
            nn.Conv2d(nf * 2, nf * 2, 3, 1, 1), #k3n128s1
            nn.InstanceNorm2d(nf * 2),
            nn.ReLU(True),
            nn.Conv2d(nf * 2, nf * 4, 3, 2, 1), #k3n256s1
            nn.Conv2d(nf * 4, nf * 4, 3, 1, 1), #k3n256s1
            nn.InstanceNorm2d(nf * 4),
            nn.ReLU(True),
        )

        self.resnet_blocks = []
        for i in range(nb):
            self.resnet_blocks.append(resnet_block(nf * 4, 3, 1, 1))

        self.resnet_blocks = nn.Sequential(*self.resnet_blocks)

        self.up_convs = nn.Sequential(
            nn.ConvTranspose2d(nf * 4, nf * 2, 3, 2, 1, 1), #k3n128s1/2
            nn.Conv2d(nf * 2, nf * 2, 3, 1, 1), #k3n128s1
            nn.InstanceNorm2d(nf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(nf * 2, nf, 3, 2, 1, 1), #k3n64s1/2
            nn.Conv2d(nf, nf, 3, 1, 1), #k3n64s1
            nn.InstanceNorm2d(nf),
            nn.ReLU(True),
            nn.Conv2d(nf, out_nc, 7, 1, 3), #k7n3s1
            nn.Tanh(),
        )

        initialize_weights(self)

    # forward method
    def forward(self, input):
        x = self.down_convs(input)
        x = self.resnet_blocks(x)
        output = self.up_convs(x)

        return output


class discriminator(nn.Module):
    # initializers
    def __init__(self, in_nc, out_nc, nf=32):
        super(discriminator, self).__init__()
        self.input_nc = in_nc
        self.output_nc = out_nc
        self.nf = nf
        self.convs = nn.Sequential(
            nn.Conv2d(in_nc, nf, 3, 1, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf, nf * 2, 3, 2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf * 2, nf * 4, 3, 1, 1),
            nn.InstanceNorm2d(nf * 4),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf * 4, nf * 4, 3, 2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf * 4, nf * 8, 3, 1, 1),
            nn.InstanceNorm2d(nf * 8),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf * 8, nf * 8, 3, 1, 1),
            nn.InstanceNorm2d(nf * 8),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(nf * 8, out_nc, 3, 1, 1),
            nn.Sigmoid(),
        )

        initialize_weights(self)

    # forward method
    def forward(self, input):
        # input = torch.cat((input1, input2), 1)
        output = self.convs(input)

        return output


class VGG19(nn.Module):
    def __init__(self, init_weights=None, feature_mode=False, batch_norm=False, num_classes=1000):
        super(VGG19, self).__init__()
        self.cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
        self.init_weights = init_weights
        self.feature_mode = feature_mode
        self.batch_norm = batch_norm
        self.num_clases = num_classes
        self.features = self.make_layers(self.cfg, batch_norm)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if not init_weights == None:
            self.load_state_dict(torch.load(init_weights))

    def make_layers(self, cfg, batch_norm=False):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

    def forward(self, x):
        if self.feature_mode:
            module_list = list(self.features.modules())
            for l in module_list[1:27]:                 # conv4_4
                x = l(x)
        if not self.feature_mode:
            x = x.view(x.size(0), -1)
            x = self.classifier(x)

        return x

In [ ]:


def edge_promoting(root, save):
    file_list = os.listdir(root)
    if not os.path.isdir(save):
        os.makedirs(save)
    kernel_size = 5
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    gauss = cv2.getGaussianKernel(kernel_size, 0)
    gauss = gauss * gauss.transpose(1, 0)
    n = 1
    for f in tqdm(file_list):
        rgb_img = cv2.imread(os.path.join(root, f))
        gray_img = cv2.imread(os.path.join(root, f), 0)
        rgb_img = cv2.resize(rgb_img, (256, 256))
        pad_img = np.pad(rgb_img, ((2,2), (2,2), (0,0)), mode='reflect')
        gray_img = cv2.resize(gray_img, (256, 256))
        edges = cv2.Canny(gray_img, 100, 200)
        dilation = cv2.dilate(edges, kernel)

        gauss_img = np.copy(rgb_img)
        idx = np.where(dilation != 0)
        for i in range(np.sum(dilation != 0)):
            gauss_img[idx[0][i], idx[1][i], 0] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 0], gauss))
            gauss_img[idx[0][i], idx[1][i], 1] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 1], gauss))
            gauss_img[idx[0][i], idx[1][i], 2] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 2], gauss))

        result = np.concatenate((rgb_img, gauss_img), 1)

        cv2.imwrite(os.path.join(save, str(n) + '.png'), result)
        n += 1

In [ ]:
def main(argv):

  parser = argparse.ArgumentParser()
  parser.add_argument('--name', required=False, default='project_name',  help='')
  parser.add_argument('--src_data', required=False, default='src_data_path',  help='sec data path')
  parser.add_argument('--tgt_data', required=False, default='tgt_data_path',  help='tgt data path')
  parser.add_argument('--vgg_model', required=False, default='pre_trained_VGG19_model_path/vgg19.pth', help='pre-trained VGG19 model path')
  parser.add_argument('--in_ngc', type=int, default=3, help='input channel for generator')
  parser.add_argument('--out_ngc', type=int, default=3, help='output channel for generator')
  parser.add_argument('--in_ndc', type=int, default=3, help='input channel for discriminator')
  parser.add_argument('--out_ndc', type=int, default=1, help='output channel for discriminator')
  parser.add_argument('--batch_size', type=int, default=8, help='batch size')
  parser.add_argument('--ngf', type=int, default=64)
  parser.add_argument('--ndf', type=int, default=32)
  parser.add_argument('--nb', type=int, default=8, help='the number of resnet block layer for generator')
  parser.add_argument('--input_size', type=int, default=256, help='input size')
  parser.add_argument('--train_epoch', type=int, default=100)
  parser.add_argument('--pre_train_epoch', type=int, default=10)
  parser.add_argument('--lrD', type=float, default=0.0002, help='learning rate, default=0.0002')
  parser.add_argument('--lrG', type=float, default=0.0002, help='learning rate, default=0.0002')
  parser.add_argument('--con_lambda', type=float, default=10, help='lambda for content loss')
  parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
  parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
  parser.add_argument('--latest_generator_model', required=False, default='', help='the latest trained model path')
  parser.add_argument('--latest_discriminator_model', required=False, default='', help='the latest trained model path')
  args = parser.parse_args(argv)

  print('------------ Options -------------')
  for k, v in sorted(vars(args).items()):
      print('%s: %s' % (str(k), str(v)))
  print('-------------- End ----------------')

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  if torch.backends.cudnn.enabled:
      torch.backends.cudnn.benchmark = True

  # results save path
  if not os.path.isdir(os.path.join(args.name + '_results', 'Reconstruction')):
      os.makedirs(os.path.join(args.name + '_results', 'Reconstruction'))
  if not os.path.isdir(os.path.join(args.name + '_results', 'Transfer')):
      os.makedirs(os.path.join(args.name + '_results', 'Transfer'))

  # edge-promoting
  if not os.path.isdir(os.path.join('data', args.tgt_data, 'pair')):
      print('edge-promoting start!!')
      edge_promoting(os.path.join('data', args.tgt_data, 'train'), os.path.join('data', args.tgt_data, 'pair'))
  else:
      print('edge-promoting already done')

  # data_loader
  src_transform = transforms.Compose([
          transforms.Resize((args.input_size, args.input_size)),
          transforms.ToTensor(),
          transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
  ])
  tgt_transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
  ])
  train_loader_src = data_load(os.path.join('data', args.src_data), 'train', src_transform, args.batch_size, shuffle=True, drop_last=True)
  train_loader_tgt = data_load(os.path.join('data', args.tgt_data), 'pair', tgt_transform, args.batch_size, shuffle=True, drop_last=True)
  test_loader_src = data_load(os.path.join('data', args.src_data), 'test', src_transform, 1, shuffle=True, drop_last=True)

  # network
  G = generator(args.in_ngc, args.out_ngc, args.ngf, args.nb)
  if args.latest_generator_model != '':
      if torch.cuda.is_available():
          G.load_state_dict(torch.load(args.latest_generator_model))
      else:
          # cpu mode
          G.load_state_dict(torch.load(args.latest_generator_model, map_location=lambda storage, loc: storage))

  D = discriminator(args.in_ndc, args.out_ndc, args.ndf)
  if args.latest_discriminator_model != '':
      if torch.cuda.is_available():
          D.load_state_dict(torch.load(args.latest_discriminator_model))
      else:
          D.load_state_dict(torch.load(args.latest_discriminator_model, map_location=lambda storage, loc: storage))
  VGG = VGG19(init_weights=args.vgg_model, feature_mode=True)
  G.to(device)
  D.to(device)
  VGG.to(device)
  G.train()
  D.train()
  VGG.eval()
  print('---------- Networks initialized -------------')
  print_network(G)
  print_network(D)
  print_network(VGG)
  print('-----------------------------------------------')

  # loss
  BCE_loss = nn.BCELoss().to(device)
  L1_loss = nn.L1Loss().to(device)

  # Adam optimizer
  G_optimizer = optim.Adam(G.parameters(), lr=args.lrG, betas=(args.beta1, args.beta2))
  D_optimizer = optim.Adam(D.parameters(), lr=args.lrD, betas=(args.beta1, args.beta2))
  G_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=G_optimizer, milestones=[args.train_epoch // 2, args.train_epoch // 4 * 3], gamma=0.1)
  D_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=D_optimizer, milestones=[args.train_epoch // 2, args.train_epoch // 4 * 3], gamma=0.1)

  pre_train_hist = {}
  pre_train_hist['Recon_loss'] = []
  pre_train_hist['per_epoch_time'] = []
  pre_train_hist['total_time'] = []

  """ Pre-train reconstruction """
  if args.latest_generator_model == '':
      print('Pre-training start!')
      start_time = time.time()
      for epoch in range(args.pre_train_epoch):
          epoch_start_time = time.time()
          Recon_losses = []
          for x, _ in train_loader_src:
              x = x.to(device)

              # train generator G
              G_optimizer.zero_grad()

              x_feature = VGG((x + 1) / 2)
              G_ = G(x)
              G_feature = VGG((G_ + 1) / 2)

              Recon_loss = 10 * L1_loss(G_feature, x_feature.detach())
              Recon_losses.append(Recon_loss.item())
              pre_train_hist['Recon_loss'].append(Recon_loss.item())

              Recon_loss.backward()
              G_optimizer.step()

          per_epoch_time = time.time() - epoch_start_time
          pre_train_hist['per_epoch_time'].append(per_epoch_time)
          print('[%d/%d] - time: %.2f, Recon loss: %.3f' % ((epoch + 1), args.pre_train_epoch, per_epoch_time, torch.mean(torch.FloatTensor(Recon_losses))))

      total_time = time.time() - start_time
      pre_train_hist['total_time'].append(total_time)
      with open(os.path.join(args.name + '_results',  'pre_train_hist.pkl'), 'wb') as f:
          pickle.dump(pre_train_hist, f)

      with torch.no_grad():
          G.eval()
          for n, (x, _) in enumerate(train_loader_src):
              x = x.to(device)
              G_recon = G(x)
              result = torch.cat((x[0], G_recon[0]), 2)
              path = os.path.join(args.name + '_results', 'Reconstruction', args.name + '_train_recon_' + str(n + 1) + '.png')
              plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
              if n == 4:
                  break

          for n, (x, _) in enumerate(test_loader_src):
              x = x.to(device)
              G_recon = G(x)
              result = torch.cat((x[0], G_recon[0]), 2)
              path = os.path.join(args.name + '_results', 'Reconstruction', args.name + '_test_recon_' + str(n + 1) + '.png')
              plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
              if n == 4:
                  break
  else:
      print('Load the latest generator model, no need to pre-train')


  train_hist = {}
  train_hist['Disc_loss'] = []
  train_hist['Gen_loss'] = []
  train_hist['Con_loss'] = []
  train_hist['per_epoch_time'] = []
  train_hist['total_time'] = []
  print('training start!')
  start_time = time.time()
  real = torch.ones(args.batch_size, 1, args.input_size // 4, args.input_size // 4).to(device)
  fake = torch.zeros(args.batch_size, 1, args.input_size // 4, args.input_size // 4).to(device)
  for epoch in range(args.train_epoch):
      epoch_start_time = time.time()
      G.train()
      G_scheduler.step()
      D_scheduler.step()
      Disc_losses = []
      Gen_losses = []
      Con_losses = []
      for (x, _), (y, _) in zip(train_loader_src, train_loader_tgt):
          e = y[:, :, :, args.input_size:]
          y = y[:, :, :, :args.input_size]
          x, y, e = x.to(device), y.to(device), e.to(device)

          # train D
          D_optimizer.zero_grad()

          D_real = D(y)
          D_real_loss = BCE_loss(D_real, real)

          G_ = G(x)
          D_fake = D(G_)
          D_fake_loss = BCE_loss(D_fake, fake)

          D_edge = D(e)
          D_edge_loss = BCE_loss(D_edge, fake)

          Disc_loss = D_real_loss + D_fake_loss + D_edge_loss
          Disc_losses.append(Disc_loss.item())
          train_hist['Disc_loss'].append(Disc_loss.item())

          Disc_loss.backward()
          D_optimizer.step()

          # train G
          G_optimizer.zero_grad()

          G_ = G(x)
          D_fake = D(G_)
          D_fake_loss = BCE_loss(D_fake, real)

          x_feature = VGG((x + 1) / 2)
          G_feature = VGG((G_ + 1) / 2)
          Con_loss = args.con_lambda * L1_loss(G_feature, x_feature.detach())

          Gen_loss = D_fake_loss + Con_loss
          Gen_losses.append(D_fake_loss.item())
          train_hist['Gen_loss'].append(D_fake_loss.item())
          Con_losses.append(Con_loss.item())
          train_hist['Con_loss'].append(Con_loss.item())

          Gen_loss.backward()
          G_optimizer.step()


      per_epoch_time = time.time() - epoch_start_time
      train_hist['per_epoch_time'].append(per_epoch_time)
      print(
      '[%d/%d] - time: %.2f, Disc loss: %.3f, Gen loss: %.3f, Con loss: %.3f' % ((epoch + 1), args.train_epoch, per_epoch_time, torch.mean(torch.FloatTensor(Disc_losses)),
          torch.mean(torch.FloatTensor(Gen_losses)), torch.mean(torch.FloatTensor(Con_losses))))

      if epoch % 2 == 1 or epoch == args.train_epoch - 1:
          with torch.no_grad():
              G.eval()
              for n, (x, _) in enumerate(train_loader_src):
                  x = x.to(device)
                  G_recon = G(x)
                  result = torch.cat((x[0], G_recon[0]), 2)
                  path = os.path.join(args.name + '_results', 'Transfer', str(epoch+1) + '_epoch_' + args.name + '_train_' + str(n + 1) + '.png')
                  plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
                  if n == 4:
                      break

              for n, (x, _) in enumerate(test_loader_src):
                  x = x.to(device)
                  G_recon = G(x)
                  result = torch.cat((x[0], G_recon[0]), 2)
                  path = os.path.join(args.name + '_results', 'Transfer', str(epoch+1) + '_epoch_' + args.name + '_test_' + str(n + 1) + '.png')
                  plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
                  if n == 4:
                      break

              torch.save(G.state_dict(), os.path.join(args.name + '_results', 'generator_latest.pkl'))
              torch.save(D.state_dict(), os.path.join(args.name + '_results', 'discriminator_latest.pkl'))

  total_time = time.time() - start_time
  train_hist['total_time'].append(total_time)

  print("Avg one epoch time: %.2f, total %d epochs time: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_time'])), args.train_epoch, total_time))
  print("Training finish!... save training results")

  torch.save(G.state_dict(), os.path.join(args.name + '_results',  'generator_param.pkl'))
  torch.save(D.state_dict(), os.path.join(args.name + '_results',  'discriminator_param.pkl'))
  with open(os.path.join(args.name + '_results',  'train_hist.pkl'), 'wb') as f:
      pickle.dump(train_hist, f)

In [ ]:
%cd /content/drive/MyDrive/CMU/SP21/18786-IDL/11785-final-project/exp_res
main([
    "--name", "bridge-flame",
    "--src_data", "/content/11785-project/data/src_cartoonflame", 
    "--tgt_data", "/content/11785-project/data/tgt_shinkai",
    "--vgg_model","/content/drive/MyDrive/CMU/SP21/18786-IDL/11785-final-project/vgg19-dcbb9e9d.pth",
    "--train_epoch", "200"])
%cd /content/

/content/drive/MyDrive/CMU/SP21/18786-IDL/11785-final-project/exp_res
------------ Options -------------
batch_size: 8
beta1: 0.5
beta2: 0.999
con_lambda: 10
in_ndc: 3
in_ngc: 3
input_size: 256
latest_discriminator_model: 
latest_generator_model: 
lrD: 0.0002
lrG: 0.0002
name: bridge-flame
nb: 8
ndf: 32
ngf: 64
out_ndc: 1
out_ngc: 3
pre_train_epoch: 10
src_data: /content/11785-project/data/src_cartoonflame
tgt_data: /content/11785-project/data/tgt_shinkai
train_epoch: 200
vgg_model: /content/drive/MyDrive/CMU/SP21/18786-IDL/11785-final-project/vgg19-dcbb9e9d.pth
-------------- End ----------------
edge-promoting already done
---------- Networks initialized -------------
generator(
  (down_convs): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): C

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1/200] - time: 104.79, Disc loss: 1.634, Gen loss: 2.046, Con loss: 5.327
[2/200] - time: 104.84, Disc loss: 1.626, Gen loss: 2.577, Con loss: 5.268
[3/200] - time: 104.67, Disc loss: 1.598, Gen loss: 2.291, Con loss: 5.345
[4/200] - time: 104.85, Disc loss: 1.384, Gen loss: 2.320, Con loss: 5.200
[5/200] - time: 104.64, Disc loss: 1.001, Gen loss: 2.492, Con loss: 5.273
[6/200] - time: 104.84, Disc loss: 0.849, Gen loss: 2.806, Con loss: 5.771
[7/200] - time: 104.65, Disc loss: 0.842, Gen loss: 2.758, Con loss: 6.251
[8/200] - time: 104.85, Disc loss: 0.730, Gen loss: 2.933, Con loss: 6.071
[9/200] - time: 104.66, Disc loss: 0.748, Gen loss: 3.007, Con loss: 6.080
[10/200] - time: 104.85, Disc loss: 0.572, Gen loss: 3.301, Con loss: 6.426
[11/200] - time: 104.65, Disc loss: 0.638, Gen loss: 3.210, Con loss: 6.464
[12/200] - time: 104.84, Disc loss: 0.542, Gen loss: 3.416, Con loss: 6.221
[13/200] - time: 104.66, Disc loss: 0.524, Gen loss: 3.643, Con loss: 6.078
[14/200] - time: 104.